In [2]:
import sys
sys.path.append('../build')
import libry as ry
import numpy as np
import time

In [4]:
C = ry.Config()
C.addFile("../scenarios/pandasTable.g")
V = ry.ConfigurationViewer()
V.setConfiguration(C)

q0 = C.getJointState()
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

## Exercise 1a) handover

In [5]:
IK = C.komo_IK(True)
IK.addObjective(feature = ry.FS.accumulatedCollisions, type = ry.OT.ineq)
IK.addObjective(feature = ry.FS.qItself, frames = ["L_finger1", "R_finger1"], type = ry.OT.eq);
IK.addObjective(feature = ry.FS.distance, frames = ["R_gripperCenter", "L_gripperCenter"], type = ry.OT.eq);
IK.addObjective(feature = ry.FS.scalarProductXX, frames = ["R_gripperCenter", "L_gripperCenter"], type = ry.OT.eq);
IK.addObjective(feature = ry.FS.scalarProductZZ, frames = ["R_gripperCenter", "L_gripperCenter"], type = ry.OT.eq, target = [-1]);

IK.optimize()
IK.getReport()

[{'x_dim': 16, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': True},
 {'order': 1.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [-1, 0],
  'sos_sumOfSqr': 0.04699128688432455},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [0],
  'eq_sumOfAbs': 0.0},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [0],
  'inEq_sumOfPos': 0.0},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'qItself-L_finger1-R_finger1',
  'vars': [0],
  'eq_sumOfAbs': 2.3687058107751858e-05},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'PairCollision-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'eq_sumOfAbs': 0.001716054485487222},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'Default-0-vecAlign-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'eq_sumOfAbs': 1.8471072434689262e-05},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'Default-0-vecAlign-R_gripperCenter-L_gripperCenter',
  'vars': [0],
  'eq_sumOfAbs': 0.002057390076772747}]

In [7]:
C.setFrameState( IK.getConfiguration(0) )
q1 = C.getJointState()
V.setConfiguration(C)
C.setJointState(q0)

## Exercise 1b) grasps the box

In [8]:
obj = C.addFrame("object")
obj.setPosition([0.,0,1.5])
obj.setQuaternion([1,0,0,0])
obj.setShape(ry.ST.ssBox, [.2,.1,.05,.02])
obj.setColor([1,0.5,0.5])
obj.setContact(1)
V.setConfiguration(C)

In [9]:
IK2 = C.komo_IK(True)
IK2.addObjective(feature = ry.FS.accumulatedCollisions, type = ry.OT.ineq, scale = [1e2])
IK2.addObjective(feature = ry.FS.qItself, frames = ["R_finger1", "L_finger1"], type = ry.OT.eq, scale = [1e2]);
IK2.addObjective(feature = ry.FS.positionDiff, frames = ["R_gripperCenter", "object"], type = ry.OT.sos);
IK2.addObjective(feature = ry.FS.distance, frames = ["R_gripperCenter", "object"], type = ry.OT.ineq, scale = [-1e2]);
IK2.addObjective(feature = ry.FS.scalarProductXX, frames = ["R_gripperCenter", "object"], type = ry.OT.eq);
IK2.addObjective(feature = ry.FS.scalarProductXY, frames = ["R_gripperCenter", "object"], type = ry.OT.eq);

IK2.optimize()
IK2.getReport()

[{'x_dim': 16, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': True},
 {'order': 1.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [-1, 0],
  'sos_sumOfSqr': 0.014371427223851051},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [0],
  'eq_sumOfAbs': 0.0},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [0],
  'inEq_sumOfPos': 0.0},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'qItself-R_finger1-L_finger1',
  'vars': [0],
  'eq_sumOfAbs': 2.3948904558901914e-30},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-object',
  'vars': [0],
  'sos_sumOfSqr': 0.003068090415975622},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'PairCollision-R_gripperCenter-object',
  'vars': [0],
  'inEq_sumOfPos': 0.0},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'Default-0-vecAlign-R_gripperCenter-object',
  'vars': [0],
  'eq_sumOfAbs': 2.6572610720587697e-05},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'Default-0-v

In [10]:
C.setFrameState( IK2.getConfiguration(0) )
q2 = C.getJointState()
V.setConfiguration(C)
C.setJointState(q0)

## Exercise 2a,b) compute/execute a motion profile

In [11]:
S = C.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [12]:
tau = .01
T = 20
motionProfile = np.zeros((T,q0.shape[0]))
for t in range(T):
    theta = -np.pi/2 + t/(T-1)*np.pi
#     motionProfile[t] = q0 + (q1-q0)*(0.5+0.5*np.sin(theta))
    motionProfile[t] = q0 + (q2-q0)*(0.5+0.5*np.sin(theta))
    
for t in range(T):
    q = motionProfile[t]
    S.step(q, tau, ry.ControlMode.position)
    time.sleep(0.1)

## Exercise 2c) path optimization

In [15]:
C.setJointState(q0)
komo = C.komo_path(1, T, T*tau, True)
komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, target=q1, scale=[1e2], order=0)
# komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, target=q2, scale=[1e2], order=0)
komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, scale=[1e2], order=1)
komo.addObjective(feature=ry.FS.accumulatedCollisions, type=ry.OT.ineq, scale=[1e2], target=[-0.02])
komo.optimize()
komo.getReport()

for t in range(T):
    C.setFrameState(komo.getConfiguration(t))
    q = C.getJointState()
    S.step(q, tau, ry.ControlMode.position)
    time.sleep(0.1)

In [ ]:
# V2 = komo.view()
# V2.playVideo()